In [21]:
from pymongo import MongoClient
#Se conecta con la base de datos creada anteriormente
MONGO_URI = 'mongodb://localhost'
client = MongoClient(MONGO_URI)
db = client.KevinMedina_FUSA
collection = db['Audios']



In [22]:
'''
    Dias almacenados
    "12/10/2020"
    "13/12/2019"
    "2/3/2021"
    "7/4/2021"
    
'''
#Se crea el mapa
import folium
import base64
import gridfs
import wave
from io import BytesIO
import playsound

m = folium.Map(location=[-39.83175223883045, -73.23092646479488], zoom_start = 13)

######################
fecha = "2/3/2021"  # <- Cambiar para detectar dia especifico
######################

fs = gridfs.GridFS(db)

#Se busca todos los audios que tengan la fecha de grabacion igual a la fecha
for audios in collection.find({"fecha_grabacion": fecha}):
    lng = audios["longitud"]
    lat = audios["latitud"]
    
    #Se decodifica el audio
    file = fs.find_one({"filename":audios["audio"]["filename"]})
    bytedata = file.read()
    aud_IO = base64.b64decode(bytedata)

    dir = "reconAUD/ConsultaA/"+ audios["audio"]["filename"]

    #Se reconstruye el audio en cuestion
    with wave.open(dir + ".wav", "wb") as audiofile:
        audiofile.setsampwidth(2)
        audiofile.setnchannels(1)
        audiofile.setframerate(44100)
        audiofile.writeframesraw(aud_IO)
    

    png = 'doge.jpg'
    encoded = base64.b64encode(open(png, 'rb').read())


    ###-> Descomentar para reproducir sonidos
    #playsound.playsound(dir + ".wav")


    #Formato html del audio
    html = f""" 
         <h1> {audios["url_audio"].split('/')[1]}  </h1>
         <p> ID: {audios["_id"]}
         <p> Fecha_grabacion: {audios["fecha_grabacion"]} <p>
         <img src="data:image/jpg;base64,{encoded.decode('UTF-8')}">
        </center>   
    """

    iframe = folium.IFrame(html, width=320, height= 200)
    popup = folium.Popup(iframe, max_width=2650)
    
    tooltip = audios["url_audio"].split('/')[1]
    folium.Marker([lng, lat], popup=popup, tooltip=tooltip).add_to(m)
m

In [31]:
m = folium.Map(location=[-39.83175223883045, -73.23092646479488], zoom_start = 13)

'''
    Humanos
    Musica
    Animales
    Climaticos y Medio ambientales
    Mecanicos
    Vehiculos
    Alertas
'''

####################################
#Cambiar esta variable, es la que depende la condicion OR
nombres_fuentes = [{"etiquetas.nombre_fuente": "Mecanicos"}, 
                    {"etiquetas.nombre_fuente": "Animales"}, 
                    {"etiquetas.nombre_fuente": "Alertas"}]
####################################

fs = gridfs.GridFS(db)

#Se busca todos los audios que tengan la fecha de grabacion igual a la fecha
for audios in collection.find({"segmentos": {"$elemMatch":{"$or": nombres_fuentes}}}):
    lng = audios["longitud"]
    lat = audios["latitud"]

    #Para que imprima todos los nombres de las etiquetas
    #y asi comprobar si efectivamente existe o no
    a = len(audios["segmentos"])
    etiquetas = ""
    for i in range(a):
        aux = audios["segmentos"][i]["etiquetas"]["nombre_fuente"]
        etiquetas = etiquetas + aux + "\n"

    dir = "reconAUD/ConsultaB/"+ audios["audio"]["filename"]
    #Se reconstruye el audio en cuestion
    with wave.open(dir + ".wav", "wb") as audiofile:
        audiofile.setsampwidth(2)
        audiofile.setnchannels(1)
        audiofile.setframerate(44100)
        audiofile.writeframesraw(aud_IO)
    ###-> Descomentar para reproducir sonidos
    #playsound.playsound("ingrese nombre" + ".wav")

    #formato HTML del frame
    html = f"""
         <h1> {audios["url_audio"].split('/')[1]} </h1>
          <p> ID: {audios["_id"]}
         <p> Fecha_grabacion: {audios["fecha_grabacion"]} <p>
         <p> Etiquetas: <i> {etiquetas} <i> <p>
    """


    iframe = folium.IFrame(html=html, width=250, height=170)
    popup = folium.Popup(iframe, max_width=2650)
    
    tooltip = audios["url_audio"].split('/')[1]
    folium.Marker([lng, lat], popup=popup, tooltip=tooltip).add_to(m)
m

In [32]:
'''
Sonidos importados:
    car
    catpur
    fly
    guau
    meow
    patting
    seacreature
    water
'''
sonido = "water"
playsound.playsound("Sonidos/" + sonido + ".wav")